In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt


KeyboardInterrupt



In [1]:
df = pd.read_csv('WELFake_Dataset.csv')
df.head()

NameError: name 'pd' is not defined

In [ ]:
# Plot Distribution of Data

plt.bar(label.index, label.values, color=['red','blue'])
plt.xlabel('Label')
plt.ylabel('Count')
plt.title('Disturbution of Fake vs Real News')
plt.xticks([0, 1], ["0=Real", "1=Fake"])
plt.show()


NameError: name 'label' is not defined

In [ ]:
# remove first column

df.drop(columns=['Unnamed: 0'], inplace=True, axis=1)
df.head()

In [ ]:
# count number of missing values from dataframe

df.isnull().sum()

In [ ]:
# fill null values with space

df.fillna(' ', inplace=True)
df.isnull().sum()
df.head()

In [ ]:
# Clean Text
 
import re
import nltk     
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer

stemmer = PorterStemmer()

def clean(text):
    
    clean_text = str(text)
    
    clean_text = re.sub(r'[^a-zA-Z\s]','',clean_text) # Remove punctuation, numbers, special characters
    
    clean_text = clean_text.lower()  # Convert text to lowercase

    tokens = clean_text.split() # Perform tokenization on text
    
    stemm_tokens = [stemmer.stem(word) for word in tokens] # perform stemming on the tokens
    
    stop_words = set(stopwords.words('english')) # Get stopwords
    clean_text = [word for word in stemm_tokens if word not in stop_words] # remove stopwords from list of words
    
    
    return " ".join(clean_text) # Join list of words back into a string

df['title'] = df['title'].apply(clean) # Apply clean function to title column

df.head()


In [ ]:
# Drop Text Column

df.drop(columns=['text'], inplace=True, axis=1)

df.head()

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Initialize TF-IDF Vectorizer Object
vectorizer = TfidfVectorizer()

# Fit and Transform Data
X = vectorizer.fit_transform(X)

print(X)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42, stratify=y)

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

In [ ]:
from sklearn.decomposition import TruncatedSVD

# Initalize TruncatedSVD
# Retain 1000 Features


svd = TruncatedSVD(n_components=1000)

# Apply on Train data
X_train = svd.fit_transform(X_train)

# Apply on Test data
X_test = svd.transform(X_test)

print(X_train.shape)
print(X_test.shape)

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        
        # Input to Hidden Layer
        self.input = nn.Linear(1000, 500)  # 1000 Inputs, 500 out
        self.relu1 = nn.ReLU()
        self.norm1 = nn.BatchNorm1d(500)  # Batch normalization
        self.drop1 = nn.Dropout(0.2)  # Dropout

        # Hidden Layer 1 to Hidden Layer 2
        self.hidden1 = nn.Linear(500, 250)  # 500 Inputs, 250 out
        self.relu2 = nn.ReLU()
        self.norm2 = nn.BatchNorm1d(250)  # Batch normalization
        self.drop2 = nn.Dropout(0.2)  # Dropout

        # Hidden Layer 2 to Hidden Layer 3
        self.hidden2 = nn.Linear(250, 50)  # 250 Inputs, 50 out
        self.relu3 = nn.ReLU()
        self.norm3 = nn.BatchNorm1d(50)  # Batch normalization
        self.drop3 = nn.Dropout(0.2)  # Dropout

        # Hidden Layer 3 to Output
        self.hidden3 = nn.Linear(50, 1)  # 50 Inputs, 1 out

    def forward(self, x):
        # Input Layer
        x = self.input(x)
        x = self.relu1(x)
        x = self.norm1(x)
        x = self.drop1(x)

        # Hidden Layer 1
        x = self.hidden1(x)
        x = self.relu2(x)
        x = self.norm2(x)
        x = self.drop2(x)

        # Hidden Layer 2
        x = self.hidden2(x)
        x = self.relu3(x)
        x = self.norm3(x)
        x = self.drop3(x)

        # Hidden Layer 3 (Output)
        x = self.hidden3(x)
        
        return x

# Initialize Model
model = NeuralNetwork()

# Initialize ADAM optimizer, to update weights of the model
optimize = torch.optim.Adam(model.parameters(), lr=0.01)

# Define Loss Function (Binary Cross Entropy)
error = nn.BCEWithLogitsLoss()


In [ ]:
# Convert Dataset to pytorch tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32)

In [ ]:
EPOCHS = 30

# Loop for number of Epochs
for epoch in range(EPOCHS):
    
    
    # Forward Propagation (calling the forward method of the neural network)
    outputs = model(X_train)
    # Convert to 1D to match Y_train shape
    outputs = outputs.squeeze(1)
    
    
    # Compute Loss between outputs and true labels
    loss = error(outputs, y_train)  # error is the loss function defined earlier
    
    
    # Clears gradients before computing the new gradients for the next iteration 
    optimize.zero_grad()
    
    # Backward propogation 
    # Computes gradients of the loss function with respect to the parameters
    loss.backward()
    
    # Uses the chosen optimizer (Adam) to adjust the model's weights based on the gradients calculated in the backpropagation step
    optimize.step()
    
    print(f"Loss after iteration {epoch} = {loss}")

NameError: name 'model' is not defined

In [ ]:
with torch.no_grad():     # Disable Gradient Computation
    # Perform forward pass using X_test data
    # Shape is [batch_size, 1]
    y_pred = model(X_test)  
    print(y_pred[0])
    
    # Convert y_pred output into labels (0 or 1)
    # reshape from [batch_size, 1] to [batch_size]
    # if greater than 0 (True) then class 1, less than or equal to 0 (False) then class 0
    # Turn boolean into integer values
    y_pred_labels = (y_pred.squeeze(1) > 0).int()
    print(y_pred_labels[0])
    
    # Calculate Accuracy 
    # compare predicted labels with actual labels 
    # divides by the total number of samples in y_test 
    accuracy = (y_pred_labels == y_test).sum().item() / y_test.size(0)
    print(f"Accuracy of the model is {accuracy}")

In [ ]:
from sklearn import metrics
import matplotlib.pyplot as plt

# Compute confusion matrix. Comparing true with predicyed labels
confusion_matrix = metrics.confusion_matrix(y_test, y_pred_labels)

# Display confusion matrix
cm_display = metrics.ConfusionMatrixDisplay(confusion_matrix = confusion_matrix, display_labels = [0, 1])

cm_display.plot()
plt.show()